# Spectrum Builder Interactive — Constructing Spectra from Intensities

This notebook demonstrates the `Spectrum` class, which takes line-by-line intensities (from the `Intensity` class) and builds a continuous wavelength-resolved spectrum. You control the wavelength grid, spectral resolution, and how multiple components are combined.

## What This Notebook Covers:
1. **Basic Spectrum Construction** — Create a spectrum from computed intensities with interactive wavelength and resolution controls
2. **Multi-Component Spectra** — Combine multiple intensity components with different temperatures and emitting areas
3. **Resolution Effects** — Explore how spectral resolution (R) affects line profiles and observable features

This is the final step in the low-level spectral synthesis pipeline: `MoleculeLineList` → `Intensity` → `Spectrum`.

In [ ]:
# Core libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Use the interactive widget backend for matplotlib
%matplotlib widget

# IPython widget libraries
import ipywidgets as widgets
from IPython.display import display, clear_output

# iSLAT low-level data types
from iSLAT.Modules.DataTypes import Intensity, Spectrum
from iSLAT.Modules.DataTypes.MoleculeLineList import MoleculeLineList

print("Imports successful!")
print(f"matplotlib backend: {matplotlib.get_backend()}")

Imports successful!
matplotlib backend: widget


In [ ]:
# --- Load HITRAN data and prepare Intensity calculator ---

from iSLAT.Modules.FileHandling import hitran_data_folder_path

data_dir = hitran_data_folder_path
water_par_file = str(data_dir / "data_Hitran_H2O.par")

# Load H2O line list
line_list = MoleculeLineList(
    molecule_id="H2O",
    filename=water_par_file
)

# Create intensity calculator
intensity_calc = Intensity(line_list)

print(f"Loaded molecule: {line_list.name}")
print(f"Number of lines: {len(line_list.get_wavelengths())}")
print(f"Wavelength range: {line_list.get_wavelengths().min():.2f} - {line_list.get_wavelengths().max():.2f} μm")

Loaded molecule: H2O
Number of lines: 305561
Wavelength range: 0.30 - 933.28 μm


In [3]:
# --- Interactive Spectrum Builder ---

# Output widget for the plot
plot_output = widgets.Output()

# --- Parameter Sliders ---

# Temperature
temp_slider = widgets.IntSlider(
    value=850, min=100, max=3000, step=50,
    description='Temperature (K):',
    continuous_update=False,
    style={'description_width': '140px'},
    layout=widgets.Layout(width='450px'),
)

# Column density (log scale)
log_n_slider = widgets.FloatSlider(
    value=18.0, min=14.0, max=22.0, step=0.25,
    description='log_10 N (cm^-2):',
    continuous_update=False,
    style={'description_width': '140px'},
    layout=widgets.Layout(width='450px'),
    readout_format='.2f',
)

# Emitting radius
radius_slider = widgets.FloatSlider(
    value=0.5, min=0.05, max=5.0, step=0.05,
    description='Radius (AU):',
    continuous_update=False,
    style={'description_width': '140px'},
    layout=widgets.Layout(width='450px'),
)

# Distance
distance_slider = widgets.IntSlider(
    value=160, min=10, max=500, step=10,
    description='Distance (pc):',
    continuous_update=False,
    style={'description_width': '140px'},
    layout=widgets.Layout(width='450px'),
)

# Spectral resolution
resolution_slider = widgets.IntSlider(
    value=3000, min=500, max=10000, step=500,
    description='R (λ/Δλ):',
    continuous_update=False,
    style={'description_width': '140px'},
    layout=widgets.Layout(width='450px'),
)

# Line broadening
dv_slider = widgets.FloatSlider(
    value=1.0, min=0.1, max=10.0, step=0.1,
    description='Broadening (km/s):',
    continuous_update=False,
    style={'description_width': '140px'},
    layout=widgets.Layout(width='450px'),
)

# Wavelength range
wave_range_slider = widgets.FloatRangeSlider(
    value=[10.0, 20.0],
    min=5.0, max=28.0,
    step=0.5,
    description='λ range (μm):',
    continuous_update=False,
    style={'description_width': '140px'},
    layout=widgets.Layout(width='450px'),
)

# Info display
info_label = widgets.HTML(value="<b>Spectrum will be generated...</b>")

def build_spectrum(change=None):
    """Build a new spectrum from current slider values."""
    # Get parameters
    temp = float(temp_slider.value)
    n_mol = 10 ** log_n_slider.value
    radius = radius_slider.value
    distance = distance_slider.value
    R = resolution_slider.value
    dv = dv_slider.value
    lam_min, lam_max = wave_range_slider.value
    
    # Calculate intensities
    intensity_calc.calc_intensity(t_kin=temp, n_mol=n_mol, dv=dv)
    
    # Create Spectrum object
    spectrum = Spectrum(
        lam_min=lam_min,
        lam_max=lam_max,
        dlambda=0.001,  # 0.001 μm resolution
        R=R,
        distance=distance
    )
    
    # Add intensity with emitting area
    emitting_area = np.pi * radius**2  # AU^2
    spectrum.add_intensity(intensity_calc, emitting_area)
    
    # Get flux
    wavelengths = spectrum.lamgrid
    flux_jy = spectrum.flux_jy
    
    # Update info
    peak_flux = np.max(flux_jy)
    info_label.value = (
        f"<b>H₂O Spectrum</b><br>"
        f"T={temp:.0f} K | N=10^{log_n_slider.value:.1f} cm^-2 | R={radius:.2f} AU<br>"
        f"d={distance} pc | Resolution R={R} | dv={dv:.1f} km/s<br>"
        f"Peak flux: {peak_flux:.4f} Jy | {len(wavelengths)} grid points"
    )
    
    # Create plot
    with plot_output:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(12, 5))
        ax.plot(wavelengths, flux_jy, 'b-', linewidth=0.5)
        ax.fill_between(wavelengths, 0, flux_jy, alpha=0.3, color='blue')
        ax.set_xlabel('Wavelength (μm)')
        ax.set_ylabel('Flux (Jy)')
        ax.set_title(f'$H_2O$ Spectrum (T={temp:.0f}K, N={n_mol:.0e} cm^-2, R={radius:.2f} AU, d={distance} pc)')
        ax.set_xlim(lam_min, lam_max)
        ax.set_ylim(0, peak_flux * 1.1 if peak_flux > 0 else 0.01)
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

# Connect sliders
for slider in [temp_slider, log_n_slider, radius_slider, distance_slider, 
               resolution_slider, dv_slider, wave_range_slider]:
    slider.observe(build_spectrum, names='value')

# Initial build
build_spectrum()

# Layout
left_controls = widgets.VBox([temp_slider, log_n_slider, radius_slider, dv_slider])
right_controls = widgets.VBox([distance_slider, resolution_slider, wave_range_slider])
controls = widgets.VBox([info_label, widgets.HBox([left_controls, right_controls])])

display(controls)
display(plot_output)

Output()

## Multi-Component Spectrum

The `Spectrum` class can combine multiple intensity components with different emitting areas. This simulates emission from regions at different radii.

In [4]:
# --- Multi-Component Spectrum: Two Temperature Components ---

# Output for multi-component plot
multi_output = widgets.Output()

# Component 1 sliders (Hot, inner region)
temp1_slider = widgets.IntSlider(
    value=1200, min=500, max=3000, step=50,
    description='T_1 (K):',
    continuous_update=False,
    style={'description_width': '80px'},
    layout=widgets.Layout(width='350px'),
)

log_n1_slider = widgets.FloatSlider(
    value=17.5, min=14.0, max=21.0, step=0.25,
    description='log N_1:',
    continuous_update=False,
    style={'description_width': '80px'},
    layout=widgets.Layout(width='350px'),
)

radius1_slider = widgets.FloatSlider(
    value=0.2, min=0.05, max=2.0, step=0.05,
    description='R_1 (AU):',
    continuous_update=False,
    style={'description_width': '80px'},
    layout=widgets.Layout(width='350px'),
)

# Component 2 sliders (Cooler, outer region)
temp2_slider = widgets.IntSlider(
    value=600, min=100, max=1500, step=50,
    description='T_2 (K):',
    continuous_update=False,
    style={'description_width': '80px'},
    layout=widgets.Layout(width='350px'),
)

log_n2_slider = widgets.FloatSlider(
    value=18.5, min=14.0, max=21.0, step=0.25,
    description='log N_2:',
    continuous_update=False,
    style={'description_width': '80px'},
    layout=widgets.Layout(width='350px'),
)

radius2_slider = widgets.FloatSlider(
    value=0.8, min=0.1, max=5.0, step=0.1,
    description='R_2 (AU):',
    continuous_update=False,
    style={'description_width': '80px'},
    layout=widgets.Layout(width='350px'),
)

# Toggle for showing individual components
show_components = widgets.Checkbox(
    value=True,
    description='Show individual components',
    style={'description_width': 'initial'},
)

multi_info = widgets.HTML(value="")

def build_multi_spectrum(change=None):
    """Build spectrum with two temperature components."""
    # Fixed parameters
    distance = 160
    R = 3000
    dv = 1.0
    lam_min, lam_max = 10.0, 20.0
    
    # Component 1 (hot)
    temp1 = float(temp1_slider.value)
    n_mol1 = 10 ** log_n1_slider.value
    radius1 = radius1_slider.value
    area1 = np.pi * radius1**2
    
    # Component 2 (cool)
    temp2 = float(temp2_slider.value)
    n_mol2 = 10 ** log_n2_slider.value
    radius2 = radius2_slider.value
    area2 = np.pi * radius2**2
    
    # Calculate intensities for each component
    intensity_calc.calc_intensity(t_kin=temp1, n_mol=n_mol1, dv=dv)
    
    # Create spectrum for component 1 only (for plotting separately)
    spec1 = Spectrum(lam_min=lam_min, lam_max=lam_max, dlambda=0.001, R=R, distance=distance)
    spec1.add_intensity(intensity_calc, area1)
    
    # Create combined spectrum
    spec_combined = Spectrum(lam_min=lam_min, lam_max=lam_max, dlambda=0.001, R=R, distance=distance)
    spec_combined.add_intensity(intensity_calc, area1)
    
    # Add second component
    intensity_calc.calc_intensity(t_kin=temp2, n_mol=n_mol2, dv=dv)
    
    spec2 = Spectrum(lam_min=lam_min, lam_max=lam_max, dlambda=0.001, R=R, distance=distance)
    spec2.add_intensity(intensity_calc, area2)
    
    spec_combined.add_intensity(intensity_calc, area2)
    
    # Get wavelengths and fluxes
    wavelengths = spec_combined.lamgrid
    flux_combined = spec_combined.flux_jy
    flux1 = spec1.flux_jy
    flux2 = spec2.flux_jy
    
    peak = np.max(flux_combined)
    
    multi_info.value = (
        f"<b>Two-Component Model</b><br>"
        f"<span style='color:red'>Component 1:</span> T={temp1}K, N=10^{log_n1_slider.value:.1f}, R={radius1:.2f} AU (hot/inner)<br>"
        f"<span style='color:blue'>Component 2:</span> T={temp2}K, N=10^{log_n2_slider.value:.1f}, R={radius2:.2f} AU (cool/outer)<br>"
        f"Peak combined flux: {peak:.4f} Jy"
    )
    
    with multi_output:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(12, 5))
        
        if show_components.value:
            ax.plot(wavelengths, flux1, 'r-', linewidth=0.5, alpha=0.7, label=f'Hot (T={temp1}K)')
            ax.plot(wavelengths, flux2, 'b-', linewidth=0.5, alpha=0.7, label=f'Cool (T={temp2}K)')
        
        ax.plot(wavelengths, flux_combined, 'k-', linewidth=1, label='Combined')
        ax.fill_between(wavelengths, 0, flux_combined, alpha=0.2, color='gray')
        
        ax.set_xlabel('Wavelength (μm)')
        ax.set_ylabel('Flux (Jy)')
        ax.set_title('Two-Component $H_2O$ Spectrum')
        ax.set_xlim(lam_min, lam_max)
        ax.set_ylim(0, peak * 1.15 if peak > 0 else 0.01)
        ax.legend(loc='upper right')
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

# Connect sliders
for slider in [temp1_slider, log_n1_slider, radius1_slider, 
               temp2_slider, log_n2_slider, radius2_slider, show_components]:
    slider.observe(build_multi_spectrum, names='value')

# Initial build
build_multi_spectrum()

# Layout: two columns for the two components
comp1_box = widgets.VBox([
    widgets.HTML("<b style='color:red'>Component 1 (Hot/Inner)</b>"),
    temp1_slider, log_n1_slider, radius1_slider
])
comp2_box = widgets.VBox([
    widgets.HTML("<b style='color:blue'>Component 2 (Cool/Outer)</b>"),
    temp2_slider, log_n2_slider, radius2_slider
])

controls = widgets.VBox([
    multi_info,
    widgets.HBox([comp1_box, comp2_box]),
    show_components
])

display(controls)
display(multi_output)

Output()

## Resolution Effects Explorer

Explore how spectral resolution affects the observed spectrum. Higher R values preserve more line structure, while lower R values smooth out individual lines.

In [5]:
# --- Resolution Comparison ---

res_output = widgets.Output()

# Resolution values to compare
res1_slider = widgets.IntSlider(
    value=500, min=100, max=5000, step=100,
    description='R_1 (low):',
    continuous_update=False,
    style={'description_width': '80px'},
    layout=widgets.Layout(width='300px'),
)

res2_slider = widgets.IntSlider(
    value=3000, min=500, max=10000, step=500,
    description='R_2 (high):',
    continuous_update=False,
    style={'description_width': '80px'},
    layout=widgets.Layout(width='300px'),
)

res_wave_slider = widgets.FloatRangeSlider(
    value=[14.0, 16.0],
    min=5.0, max=28.0,
    step=0.25,
    description='λ range:',
    continuous_update=False,
    style={'description_width': '80px'},
    layout=widgets.Layout(width='400px'),
)

res_info = widgets.HTML(value="")

def compare_resolutions(change=None):
    """Compare spectra at two different resolutions."""
    R1 = res1_slider.value
    R2 = res2_slider.value
    lam_min, lam_max = res_wave_slider.value
    
    # Fixed parameters
    temp = 850
    n_mol = 1e18
    radius = 0.5
    distance = 160
    dv = 1.0
    area = np.pi * radius**2
    
    intensity_calc.calc_intensity(t_kin=temp, n_mol=n_mol, dv=dv)
    
    # Low resolution spectrum
    spec_low = Spectrum(lam_min=lam_min, lam_max=lam_max, dlambda=0.0005, R=R1, distance=distance)
    spec_low.add_intensity(intensity_calc, area)
    
    # High resolution spectrum
    spec_high = Spectrum(lam_min=lam_min, lam_max=lam_max, dlambda=0.0005, R=R2, distance=distance)
    spec_high.add_intensity(intensity_calc, area)
    
    res_info.value = (
        f"<b>Resolution Comparison</b><br>"
        f"Fixed: T={temp}K, N={n_mol:.0e} cm^-2, R={radius} AU, d={distance} pc<br>"
        f"<span style='color:blue'>Low resolution:</span> R = {R1} (Δλ ≈ {15/R1*1000:.1f} nm at 15μm)<br>"
        f"<span style='color:red'>High resolution:</span> R = {R2} (Δλ ≈ {15/R2*1000:.1f} nm at 15μm)"
    )
    
    with res_output:
        clear_output(wait=True)
        fig, axes = plt.subplots(2, 1, figsize=(12, 7), sharex=True)
        
        peak = max(np.max(spec_low.flux_jy), np.max(spec_high.flux_jy))
        
        axes[0].plot(spec_low.lamgrid, spec_low.flux_jy, 'b-', linewidth=0.5)
        axes[0].fill_between(spec_low.lamgrid, 0, spec_low.flux_jy, alpha=0.3, color='blue')
        axes[0].set_ylabel('Flux (Jy)')
        axes[0].set_title(f'Low Resolution: R = {R1}')
        axes[0].set_ylim(0, peak * 1.1)
        axes[0].grid(True, alpha=0.3)
        
        axes[1].plot(spec_high.lamgrid, spec_high.flux_jy, 'r-', linewidth=0.5)
        axes[1].fill_between(spec_high.lamgrid, 0, spec_high.flux_jy, alpha=0.3, color='red')
        axes[1].set_xlabel('Wavelength (μm)')
        axes[1].set_ylabel('Flux (Jy)')
        axes[1].set_title(f'High Resolution: R = {R2}')
        axes[1].set_xlim(lam_min, lam_max)
        axes[1].set_ylim(0, peak * 1.1)
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# Connect
res1_slider.observe(compare_resolutions, names='value')
res2_slider.observe(compare_resolutions, names='value')
res_wave_slider.observe(compare_resolutions, names='value')

# Initial
compare_resolutions()

display(widgets.VBox([res_info, widgets.HBox([res1_slider, res2_slider]), res_wave_slider]))
display(res_output)

Output()